In [1]:
import numpy as np
import pandas as pd
import os
from googlesearch import search
import re
import math

In [2]:
df = pd.read_excel('/Users/John/Desktop/DS340W/data/stopandfrisk.xlsx')

In [4]:
# 'STOP_LOCATION_PRECINCT'
df2 = df[['SUSPECT_ARRESTED_FLAG', 'SUMMONS_ISSUED_FLAG', 'FRISKED_FLAG', 'STOP_LOCATION_FULL_ADDRESS', 'STOP_LOCATION_ZIP_CODE' ,'STOP_LOCATION_BORO_NAME']]
df2.head()

,SUSPECT_ARRESTED_FLAG,SUMMONS_ISSUED_FLAG,FRISKED_FLAG,STOP_LOCATION_FULL_ADDRESS,STOP_LOCATION_ZIP_CODE,STOP_LOCATION_BORO_NAME
0,Y,N,Y,248 E 83 ST,10028,MANHATTAN
1,N,N,Y,248 E 83 ST,10028,MANHATTAN
2,N,N,Y,248 E 83 ST,10028,MANHATTAN
3,Y,N,Y,LINDEN BLVD && SHEPHERD AVE,11208,BROOKLYN
4,N,N,N,481 5 AVE,11215,BROOKLYN


In [11]:
zipcode_counts = df2.groupby('STOP_LOCATION_ZIP_CODE').size().reset_index(name='num_stopandfrisk')
zipcode_counts = zipcode_counts.rename(columns={'STOP_LOCATION_ZIP_CODE': 'zipcode'})
zipcode_counts.head()

,zipcode,num_stopandfrisk
0,10000,6
1,10001,92
2,10002,109
3,10003,82
4,10004,7


In [6]:
nypd_complaint = pd.read_csv('/Users/John/Desktop/DS340W/data/NYPD_Complaint_Data_Final.csv', dtype={'zipcode': 'object'})

/var/folders/r6/qcs_flrd7fj2bg8ml8rp61lc0000gn/T/ipykernel_64667/871280548.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  nypd_complaint = pd.read_csv('/Users/John/Desktop/DS340W/data/NYPD_Complaint_Data_Final.csv', dtype={'zipcode': 'object'})


In [7]:
nypd_complaint.head()

,Unnamed: 0.1,Unnamed: 0,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,...,Latitude,Longitude,CMPLNT_FR_YEAR,CMPLNT_FR_MONTH,CMPLNT_FR_DAY,geometry,pop_est,zipcode,num_policestation,num_restroom
0,0,0,280510523,14,MANHATTAN,01/11/2024,04:00:00,NaN,NaN,COMPLETED,...,40.753533,-73.994537,2024,1,11,POINT (-73.9945368920152 40.7535327012632),9687.0,10018,1.0,0.0
1,1,1,286736978,122,STATEN ISLAND,01/01/2015,13:00:00,01/01/2015,15:00:00,COMPLETED,...,40.574286,-74.105914,2015,1,1,POINT (-74.1059144051212 40.5742856881925),53142.0,10306,1.0,9.0
2,2,2,289019347,1,MANHATTAN,06/20/2024,16:12:00,06/20/2024,16:13:00,COMPLETED,...,40.709556,-74.006435,2024,6,20,POINT (-74.00643483276845 40.70955616028041),22800.0,10038,1.0,5.0
3,3,3,283688417,121,STATEN ISLAND,03/11/2024,10:00:00,03/13/2024,13:20:00,COMPLETED,...,40.634542,-74.165142,2024,3,11,POINT (-74.165142 40.634542),25947.0,10303,0.0,2.0
4,4,4,280934552,122,STATEN ISLAND,11/14/2023,12:00:00,11/17/2023,12:00:00,COMPLETED,...,40.537874,-74.155293,2023,11,14,POINT (-74.155293 40.537874),61741.0,10312,0.0,7.0


In [13]:
df_merged = pd.merge(nypd_complaint, zipcode_counts, on='zipcode', how='left')
df_merged['num_stopandfrisk'] = df_merged['num_stopandfrisk'].fillna(0)
df_merged = df_merged.drop(['Unnamed: 0.1', 'Unnamed: 0',], axis = 1)
df_merged.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURISDICTION_CODE,JURIS_DESC,...,Longitude,CMPLNT_FR_YEAR,CMPLNT_FR_MONTH,CMPLNT_FR_DAY,geometry,pop_est,zipcode,num_policestation,num_restroom,num_stopandfrisk
0,280510523,14,MANHATTAN,01/11/2024,04:00:00,NaN,NaN,COMPLETED,0,N.Y. POLICE DEPT,...,-73.994537,2024,1,11,POINT (-73.9945368920152 40.7535327012632),9687.0,10018,1.0,0.0,0.0
1,286736978,122,STATEN ISLAND,01/01/2015,13:00:00,01/01/2015,15:00:00,COMPLETED,0,N.Y. POLICE DEPT,...,-74.105914,2015,1,1,POINT (-74.1059144051212 40.5742856881925),53142.0,10306,1.0,9.0,0.0
2,289019347,1,MANHATTAN,06/20/2024,16:12:00,06/20/2024,16:13:00,COMPLETED,1,N.Y. TRANSIT POLICE,...,-74.006435,2024,6,20,POINT (-74.00643483276845 40.70955616028041),22800.0,10038,1.0,5.0,0.0
3,283688417,121,STATEN ISLAND,03/11/2024,10:00:00,03/13/2024,13:20:00,COMPLETED,0,N.Y. POLICE DEPT,...,-74.165142,2024,3,11,POINT (-74.165142 40.634542),25947.0,10303,0.0,2.0,0.0
4,280934552,122,STATEN ISLAND,11/14/2023,12:00:00,11/17/2023,12:00:00,COMPLETED,0,N.Y. POLICE DEPT,...,-74.155293,2023,11,14,POINT (-74.155293 40.537874),61741.0,10312,0.0,7.0,0.0


In [14]:
df_merged.to_csv('/Users/John/Desktop/DS340W/data/NYPD_Complaint_Data_Final2.csv')